# Import all necessary libraries and .py

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd  

from load_data import *
from pred_score import *
from Filter_FS import *
from overlap_genes import *
from crossValidation import *
from evaluation_measure import *

#Fixing seed to get reproducible results
random.seed(3)
np.random.seed(3)

ModuleNotFoundError: No module named 'evaluation_measure'

# Load normalized data 

In [2]:
#Enter filepath bellow as a str
filepath = '../data/processed_data/AE3.csv' #Provide the path to the normalized data you would like to predict the family of.

norm = np.array(pd.read_csv (filepath))
print('Shape of the normalized data should be: number of cells x number of genes. ', norm.shape)

#If the optimization takes too much time, you might consider removing all genes that are not expressed in at least 5% of the cells.
#To do so, uncomment the lines bellow
gene_expressed = filter_norm_data(norm.T, 0.05)
norm = (norm.T[gene_expressed]).T

Shape of the normalized data should be: number of cells x number of genes.  (333, 11895)


# Baseline with all genes

In [4]:
#Predict families of the dataset and evaluate using Cophenetic coefficient
model = FamiliesClusters(family_interest = None, Scoring_ = compute_cophe_coeff, maximize = True)
pred = model.fit_predict(norm)

print('Cophenetic coefficient and recovery: ', model.score, ' ', model.recovery)

NameError: name 'compute_cophe_coeff' is not defined

# Mutual information maximizer (MIM)

Perform grid search to find the final size of the gene subset. You might want to play around with the range of final number of genes depending on the size of the dataset. For MIM fature selection, the number of neighbors for the computatation of the mutual information of each gene is a tunable parameter.

In [ ]:
#Define parameters for MIM method
N = np.arange(109,1500,300)
kwargs = {'Model': FamiliesClusters, 'Scoring': compute_cophe_coeff, 'maximize': True,'N': N, 'n_neighbors': 3, 'plot': True} 

#Cross validate the MIM optimization 
subset, score_training, score_testing = cross_validation(y= None, x=norm, Model_test=FamiliesClusters, Scoring_test=compute_cophe_coeff, maximize_test=True, kfold=5,  func=MIM, **kwargs)
mean_score_test, std_score_test = np.mean(score_testing), np.std(score_testing)    
print('test score and std ', mean_score_test, std_score_test)

#Predict and evaluate on whole data  set
model = FamiliesClusters(np.unique(y),compute_RI,True)
x_subset = AE3[:, subset]
pred = model.fit_predict(x_subset,y)

print("TP, FP, ratio, sensitivity, specificity, precision, NPV, FDR, FNR = ", compute_statTP(y,pred))